## Comparing a cached LDAP authorization plugin to a non-cached plugin

I set up the server to use the cached version of the plugin and used JMeter to hit the server every few seconds. The TTL on the cache was set low enough such that JMeter caused a cache miss for every few cache hit requests.

I then switched to the non-cached version of the plugin and used JMeter again to send requests to DynaFed.

I edited the rsyslog config to copy any lines that had timings in them to a seperate timing log file, which is what we have here and that we must parse to get the data. 

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from IPython.core.pylabtools import figsize
figsize(16, 4)
import plotly
plotly.offline.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt

This notebook uses plotly for it's nice interactivity features. However, this means that you can't view the graphs directly on GitHub. To view this notebook properly use nbviewer by going to http://nbviewer.jupyter.org/github/stfc/dynafed-ldap-plugin/tree/master/docs/Cached_LDAP_plugin_analysis.ipynb

In [3]:
cache_log_path = "cache_timings.log"

with open(cache_log_path, "r") as f:
    cache_log_content = f.readlines()

hit_durations = []
miss_durations = []
for line in cache_log_content:
    duration = line.split(" ")[-1]
    if "hit" in line:
        hit_durations.append(float(duration))
    elif "miss" in line:
        miss_durations.append(float(duration))

cache_hits = pd.Series(data=hit_durations)
cache_misses = pd.Series(data=miss_durations)

In [98]:
print("Hits: " + str(cache_hits.mean()) + " ± " + str(cache_hits.std()))
print("Misses: " + str(cache_misses.mean()) + " ± " + str(cache_misses.std()))
print("No of Hits/Misses: " + str(cache_hits.size) + " / " + str(cache_misses.size))

Hits: 0.0004071294223631877 ± 0.0006438440558664829
Misses: 0.037009366109955824 ± 0.07920828714706749
No of Hits/Misses: 22113 / 7118


In [102]:
no_cache_log_path = "no_cache_timings.log"

with open(no_cache_log_path, "r") as f:
    no_cache_log_content = f.readlines()

no_cache_durations = []
for line in no_cache_log_content:
    duration = line.split(" ")[-1]
    # filter out very abnormal log value of 50s
    if float(duration) < 50:
        no_cache_durations.append(float(duration))

no_cache_timings = pd.Series(data=no_cache_durations)

In [103]:
print("Stats: " + str(no_cache_timings.mean()) + " ± " + str(no_cache_timings.std()))
print("No of requests: " + str(no_cache_timings.size))

Stats: 0.014544641913713867 ± 0.04405157566116586
No of requests: 10093


Now that we have parsed the log files and know some of the initial stats, let's plot some box plots to see how the data is distributed

In [94]:
cache_hits_trace = plotly.graph_objs.Box(x=cache_hits, name="Cache Hits", boxmean='sd')
cache_miss_trace = plotly.graph_objs.Box(x=cache_misses, name="Cache Misses", boxmean='sd')
no_cache_trace = plotly.graph_objs.Box(x=no_cache_timings, name="No Cache", boxmean='sd')
traces = [no_cache_trace, cache_miss_trace, cache_hits_trace]
boxes_layout = {
    "title": "Comparison of the distributions of non-cached requests and cache hits and misses",
    "xaxis": {
        "title": "Request time (s)",
    },
}
boxes_fig = {
    "data": traces,
    "layout": boxes_layout,
}
plotly.offline.iplot(boxes_fig)

Here it can be seen that cache misses are about 2 times slower than non-cached requests. This is due to the fact that in the cached version of the plugin the LDAP requests every single attribute so that all information on the entry can be cached, whereas in the non-cached version since we send a request for every authorization attempt we can instead only request the attributes we need to check for authorization, which means that the LDAP server is faster in responding to the request since fewer atttributes need to be fetched and bundled into the response.

However, cache hits are 2 orders of magnitude smaller than both cache miss and non-cached requests. This is because if we have the LDAP information for a user then we don't need to contact the LDAP server at all, and the only reason the plugin is slow is due to the LDAP requests. Just from intuition, it is obvious that as long as we have a sufficient amount of cache hit requests to cache miss requests then it is overall much more efficient to use caching.

To optimise caching fully, we'd need to decide on a sensible TTL for the cache. Too low and the performance gain is minimal due to the amount of cache misses and too high and risk information on the LDAP server changing and our cache being stale. However, we wouldn't be expecting the information for a user in the LDAP directory to be changing all the time, and even if a user's information does change it is reasonable for that user to expect it to take some time to propogate around all the services that rely on LDAP.

This means that there is some flexibility in the cache TTL. Below are some calculations on different TTL times under different assumptions of user load to explore what value we should set TTL to.

In [75]:
def calc_average_cache_time(hit_avg, miss_avg, ttl, req_freq):
    if ttl < req_freq:
        # everything is a cache miss since TTL times out before next request is sent
        return miss_avg
    # work out how many cache hits we get per cache miss
    total_cache_hit_requests = (ttl) / req_freq
    # calculate combined average
    cached_cost = (miss_avg + total_cache_hit_requests*hit_avg) / (total_cache_hit_requests + 1)
    return cached_cost

In [74]:
# average cost per request of caching vs not caching
# assumptions
# cache TTL is 30 mins
# 1 request every 5 seconds

non_cached_cost = no_cache_timings.mean()
cache_hit_cost = cache_hits.mean()
cache_miss_cost = cache_misses.mean()

print("Non-Cached Cost: " + str(non_cached_cost))
print("Cached Cost: " + str(calc_average_cache_time(cache_hit_cost, cache_miss_cost, 1800, 5)))

Non-Cached Cost: 0.014544641913713867
Cached Cost: 0.0005085206597249401


In [90]:
# ttl times are how long a user's LDAP info is stored before it's considered stale and the next request causes a cache miss
# which refreshes the user's information
# request times are how often a single user is sending requests to our server. the more a user is sending requests
# then the more efficient it is to use caching and so a lower TTL can be used. if users only send requests infrequently
# then a higher TTL is requried for the same performance gain

ttl_times = [1,5,10,15,20,25,30,35,40,45,60,90,120,150,180,210,240,270,300,360,420,480,540,600,900,1200,1500,1800]
request_times = [1,2,5,10,20,30,60,120,300]
target = 0.005
index = pd.MultiIndex.from_product([request_times, ttl_times], names=["Time between requests", "TTL"])

cache_timings_per_ttl = []
for request_time in request_times:
    for time in ttl_times:
        cache_timings_per_ttl.append(calc_average_cache_time(cache_hit_cost,cache_miss_cost, time, request_time))

cache_ttl_data = pd.DataFrame(data={"Average cached request cost": cache_timings_per_ttl}, index=index)
cache_ttl_data

Average cached request cost
Time between requests TTL                              
1                     1                        0.018708
                      5                        0.006508
                      10                       0.003735
                      15                       0.002695
                      20                       0.002150
                      25                       0.001815
                      30                       0.001588
                      35                       0.001424
                      40                       0.001300
                      45                       0.001203
                      60                       0.001007
                      90                       0.000809
                      120                      0.000710
                      150                      0.000650
                      180                      0.000609
                      210                      0.000581
                      240                      0.000559
                      270                      0.000542
                      300                      0.000529
                      360                      0.000509
                      420                      0.000494
                      480                      0.000483
                      540                      0.000475
                      600                      0.000468
                      900                      0.000448
                      1200                     0.000438
                      1500                     0.000432
                      1800                     0.000427
2                     1                        0.037009
                      5                        0.010865
...                                                 ...
120                   1500                     0.003118
                      1800                     0.002695
300                   1                        0.037009
                      5                        0.037009
                      10                       0.037009
                      15                       0.037009
                      20                       0.037009
                      25                       0.037009
                      30                       0.037009
                      35                       0.037009
                      40                       0.037009
                      45                       0.037009
                      60                       0.037009
                      90                       0.037009
                      120                      0.037009
                      150                      0.037009
                      180                      0.037009
                      210                      0.037009
                      240                      0.037009
                      270                      0.037009
                      300                      0.018708
                      360                      0.017045
                      420                      0.015658
                      480                      0.014485
                      540                      0.013479
                      600                      0.012608
                      900                      0.009558
                      1200                     0.007728
                      1500                     0.006508
                      1800                     0.005636

[252 rows x 1 columns]

We can then plot the TTL against average request time for each of the different user request rate assumptions. We also plot a horizontal line at 0.005 (5ms), which is the maximum processing time that DynaFed allows without putting a warning in the logs.

In [91]:
d = []
for request_time in request_times:
    graph = plotly.graph_objs.Scatter(
        x = cache_ttl_data.index.get_level_values("TTL"),
        y = cache_ttl_data.xs(request_time, level="Time between requests")["Average cached request cost"],
        name = str(request_time) + " second" + ("s between requests" if request_time != 1 else " between requests")
    )
    d.append(graph)

lines_layout = {
    "title": "TTL vs Average request time for various user request rates",
    "xaxis": {
        "title": "Time to live (s)",
    },
    "yaxis": {
        "title": "Average request time (s)",
    },
    "shapes": [
    {
        "type": 'line',
        "xref": 'paper',
        "x0": 0,
        "y0": target,
        "x1": 1,
        "y1": target,
        "line": {
            "color": 'rgb(255, 0, 0)',
            "width": 1.5,
            "dash":'dash'
        }
    }
    ]
}
lines_fig = {
    'data': d,
    'layout': lines_layout,
}
plotly.offline.iplot(lines_fig)

This graph shows affirms the assumptions I made before I plotted the graph. Higher TTL obviously increases efficiency, and it follows a $\frac{1}{x}$ like distribution such that increasing TTL to higher and higher values has decreasing efficiency gains the higher the TTL. Also, the higher the time per user requests flattens the line such that a higher TTL is required to get below the 0.005s target and also that it is harder (aka requires a larger TTL increase) to get to the target value.

However, we can still implement pretty reasonable values. If a user makes requests every minute or two minutes then having a TTL of 20 minutes still gets us well below the 0.005s average per request. If a user only makes a request every 5 minutes then a larger TTL like 30 or 40 minutes is needed, but that that rate the user is accessing the site very infrequently so would hopefulyl be less annoyed by the initally "long" request.